In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
import csv
from sklearn.feature_extraction.text import CountVectorizer,  TfidfVectorizer, HashingVectorizer
from sklearn.ensemble import RandomForestClassifier 
import nltk
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
train_df.head()

In [ ]:
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
test_df.head()

In [ ]:
train_df.isnull().sum()

In [ ]:
X = train_df.drop(["id"],axis=1)
X.head()

Data PreProcessing


In [ ]:
key = X["keyword"].value_counts().index[0]
print(key)
loc = X["location"].value_counts().index[0]
print(loc)

In [ ]:
train_df['keyword'] = train_df['keyword'].fillna(train_df['keyword'].value_counts().idxmax())
train_df['location'] = train_df['location'].fillna(train_df['location'].value_counts().idxmax())
train_df.isnull().sum()

In [ ]:
X_new = train_df.drop(["target"],axis=1)
X_new.head()

In [ ]:
y = train_df["target"]
y.shape

In [ ]:
test_df.isnull().sum()

In [ ]:
test_df['keyword'] = test_df['keyword'].fillna(test_df['keyword'].value_counts().idxmax())
test_df['location'] = test_df['location'].fillna(test_df['location'].value_counts().idxmax())
test_df.isnull().sum()


Text PreProcessing

In [ ]:
X_new = X_new.drop(["id"],axis=1)
X_new.head()

In [ ]:
X_new.replace("[^a-zA-Z]", " ",regex = True, inplace = True)
X_new.head()

In [ ]:
for i in X_new.columns:
    X_new[i] = X_new[i].str.lower()
X_new.head()

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
X_new['keyword'].apply(lambda x: [item for item in x if item not in stop])
X_new['location'].apply(lambda x: [item for item in x if item not in stop])
X_new['text'].apply(lambda x: [item for item in x if item not in stop])
print(X_new.shape)

Text PreProcessing in Test Data¶


In [ ]:
ori_test = test_df.drop(["id"],axis=1)

In [ ]:
ori_test.replace("[^a-zA-Z]", " ",regex = True, inplace = True)
ori_test.head()

In [ ]:
for i in ori_test.columns:
    ori_test[i] = ori_test[i].str.lower()

ori_test['keyword'].apply(lambda x: [item for item in x if item not in stop])
ori_test['location'].apply(lambda x: [item for item in x if item not in stop])
ori_test['text'].apply(lambda x: [item for item in x if item not in stop])
print(ori_test.shape)

In [ ]:
X_new["sentence"] = X_new['keyword'] + " " + X_new['text']
train_text = np.array(X_new["sentence"])
print(train_text[0])

# test dataset
ori_test["sentence"] = ori_test['keyword'] + " " + ori_test['text']
test_text = np.array(ori_test["sentence"])
print(test_text[0])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_text,y,test_size=0.25,random_state=0)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(x_train)

keyword = vectorizer.get_feature_names()
X_train = vectorizer.transform(x_train)
X_test = vectorizer.transform(x_test)
X_test_new = vectorizer.transform(test_text)

In [ ]:
print(x_train[0])
print("\n")
print(X_train[0])

In [ ]:
rc = RandomForestClassifier(max_depth = 400 ,random_state=0, n_estimators = 300)
rc.fit(X_train,y_train) 

pred1 = rc.predict(X_test)
matrix = confusion_matrix(y_test,pred1)
print(matrix)
score = accuracy_score(y_test,pred1)
print(score)
report = classification_report(y_test,pred1)
print(report)
prediction1 = rc.predict(X_test_new)
prediction1


In [ ]:
data = {'id':test_df["id"],'target':prediction1}
output = pd.DataFrame(data, columns = ['id','target'])
output.index = test_df.index

output.to_csv("submission.csv", index = False)    


a = pd.read_csv("submission.csv")
a